# UV-K6 Digital Modulation Modification

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

Please attribute the work to *Rob Riggs, WX9O, Mobilinkd LLC*.

One of the holy grails of amateur radio is an inexpensive FM transceiver with a flat response. An old-school
ham would call this a radio with *discriminator output and varactor input* but everything is digital and
I/Q modulated these days, so *a flat response* is a clearer way to say what we are after. These are radios
that are capable of digital modes like 9600 baud GFSK or 4-FSK. Radios such as these are typically expensive
mobile units or base stations with packet data ports. These typically cost many hundreds of dollars. To date,
only Kenwood has made a hand-held radio (long since discontinued) which exposed this capability directly.

The Quansheng UV-K5 and variants (including the UV-K6) have an active hacking community. The hardware has
been well-documented and the firmware reverse-engineered. Customized firmware variants are in wide use. The
BK4819 RF IC used in this device is fairly well documented.

I have discovered that with a few trivial hardware modification and some new firmware features, we can
enhance these $30 radios to provide a flat response on both transmit and receive, allowing these radios
to be used for digital modulation. This article documents those changes and how to get started with digital
modes using this radio, along with a modem such as the Mobilinkd TNC4.

# Changelog

 - 2024-06-18 Initial publication.
 - 2024-07-04 MIC ADC update.
 - 2024-07-07 MIC capacitor (C77) value change. Notes on BK4819_REG_30 MIC ADC behavior.

# Overview of Changes

The hardware changes which are documented below allow passing low-frequency, wide, flat baseband between
the digital modem and the radio's RF chip. This is an absolute necessity for digital baseband modulation.

The firmware changes adds a new Digital (DIG) mode which does the following:

 1. Eliminate pre-emphasis on transmit and de-emphasis on receive.
 2. Eliminate the TX & RX 300Hz sub-audio channel high-pass filter which is used for CTCSS and DCS.
 4. Eliminate the TX & RX 3000Hz low-pass audio filter.
 5. Bypass the transmit low-pass filter.
 6. Disable the MIC AGC and the ALC (limiter).
 7. Disable squelch.
 8. ~~Keep MIC ADC enabled in RX mode to establish and maintain proper DC bias of the MIC input.~~
 9. Enable MIC ADC during init for a short period to establish proper DC bias of the MIC input.
 10. Reduce turn-around time from ~140ms to less than 80ms by leaving the audio amplifier enabled during
    TX. This also reduces a DC offset error at start of RX. The cost is some wasted current use during TX.

Note that the EGZUMER firmware, on which these changes are based, already reduces the turn-around time of the
stock UV-K6 firmware from over 350ms to around 140ms.

# DISCLAIMER

The hardware modifications documented here may result in permanent damage to your radio. The firmware
changes may have introduced bugs which affect other modes.

# Prequisites

You will need ~~3 10uF, 10V (or higher) 0603 MLCC capacitors~~ 2 10uF 10V (or higher) 0603 MLCC
capacitors, 1 1uF 16V 0603 MLCC capacitor, and the skill and equipment needed to do surface
mount work on 0402 parts. A tool for loosening the nuts on the antenna connector and potentiometer
(AKA [X-Key Repair Tool](https://amzn.to/3KIrhDV)) will also come in handy.

Optional components are 2 0402 0 Ohm resistors. Two 1.6mm lengths of wire will work in their place.

You will also need a programming cable to flash the firmware.

# A Caveat

The UV-K6 is a rather poor transceiver. It's input filtering is quite wide, and it suffers from the same
problem that most of the CCRs&trade; (*Cheap Chinese Radios*) have -- the receiver is subject to desense
when any strong signal exists in the passband of the receiver. That said, the Quansheng UV-K6 does
substantially better than most of the CCRs that I have tested.

# References

I am going to be referring to a few sites which have done an amazing job reversing engineering the radio's
firmware and its hardware, and in collecting documentation for the various ICs and other components used
in its construction.

My primary references are:

 * Firmware from *egzumer*: https://github.com/egzumer/uv-k5-firmware-custom/
 * Hardware from *mentalDetector*: https://github.com/mentalDetector/Quansheng_UV-K5_PCB_R51-V1.4_PCB_Reversing_Rev._0.9
 * IC Documentation from *ludwich66*: https://github.com/amnemonic/Quansheng_UV-K5_Firmware/tree/main/hardware

These may be forks of other repos. I tried to link to the repos that seem to be currently authoritative.

The firmware enhancements to support digital modualation mode is forked directly from *egzumer*. There are
currently 320 forks of this repo, some with avid users. But *egzumer* is still being maintained and is the
parent repo for most stand-alone firmware versions.

The schematics in this document are from the *mentalDetector* KiCAD schematics. They are accurate enough
to serve as a guide. There are changes between the 1.4 PCB and the 1.5 PCB in the UV-K6.

The IC documentation from *ludwich66* was instrumental in designing the modifications to the output section,
and in making the required firmware changes needed to properly configure the BK4819.

# Hardware Changes

There are 3 primary changes which need to be made, one on the audio input side to allow low frequencies in
via the MIC line, and 2 on the audio output side to allow low frequencies from the BK4819 and through the
LN4898 amplifier.

## Disassembly

There are a number of videos that show how to disassemble the radio, so I won't go into great detail here.

You will need a soldering iron to disconnect the speaker.

 1. Remove the battery.
 2. Remove the antenna.
 3. Remove the power/volume knob by pulling straight up.
 4. Remove the metal frame from the enclosure by prying the bottom of the case until the frame is released.
 5. Be careful not to damage the wires connecting the speaker to the PCB.
 6. Carefully slide the frame out by sliding it down until the potentiometer and antenna connector clear the case. Do this gently.
 7. Remove the loose plastic piece protecting the speaker-mic connector.
 8. Paying attention to the speaker wire, set the enclosure face down and the PCB side up so that the speaker wire connections are accessible.
 9. Use a soldering iron to disconnect the two speaker wires. Do not skip this step.
 10. Loosen the nuts holding the antenna connector and poteniometer to the frame. There is no need to completely remove them.
 11. Using a small screwdriver, bend inward the plastic tab on the lower left side holding the LCD to the PCB to release it.
 12. Gently fold the LCD screen up to reveal two metal screws.
 13. Remove the 5 screws holding the PCB to the frame.
 14. Lift the PCB from the frame.

## Audio Input Filter

<table>
  <tr>
    <td> <img src="Schematic-Audio-Input.png" alt="Audio input section of UV-K6 schematic diagram" style="width: 800px;"/> </td>
  </tr>
</table>

C77 is a 100nF 0402 capacitor. R69 is a 0 Ohm 0603 resistor. These parts are in the area normally covered by the LCD.

 1. Remove both C77 and R69.
 2. Short the pads of C77 with a shunt. Use a 0 Ohm 0402 resistor if you have one handy. Otherwise a 1.6mm or .040" piece of wire will work.
 3. Replace R69 with a ~~10uF~~ 1uF 16V 0603 MLCC capacitor.

<table>
  <tr>
    <td> <img src="Image-Audio-Input-Mod.png" alt="Audio input modification of UV-K6" style="width: 800px;"/> </td>
  </tr>
</table>

## LN4898 Audio Amplifier Coupling

<table>
  <tr>
    <td> <img src="Schematic-Audio-Amp.png" alt="Audio amplifier section of UV-K6 schematic diagram" style="width: 800px;"/> </td>
  </tr>
</table>

C31 is a 100nF capacitor which forms a high-pass filter as part of the input to the audio amplifier. With the
4.7k resistor at R40, this filters low frequencies below about 300Hz. These parts are in the area normally
covered by the LCD.

 1. Add two 10uF 10V capacitors in parallel with C31 as shown below.

<table>
  <tr>
    <td> <img src="Image-Audio-Amp-Mod.png" alt="Audio amplifier modification of UV-K6" style="width: 800px;"/> </td>
  </tr>
</table>

## BK4819 Output Coupling

The BK4819 output coupling capacitor is on the bottom of the PCB, facing the metal frame.

<table>
  <tr>
    <td> <img src="Schematic-BK4819-Audio-Out.png" alt="BK4819 section of UV-K6 schematic diagram" style="width: 800px;"/> </td>
  </tr>
</table>

C71 is a 100nF output coupling capacitor from the BK4819 `EARO` (earphone output) pin, feeding the audio amplifier
through the volume control potentiometer. It needs to be removed and bypassed with a 0 Ohm shunt.

 1. Remove C71.
 2. Install a 0 Ohm 0402 resistor in place of C71 if you have one handy. Otherwise a 1.6mm or .040" piece of wire will work.

<table>
  <tr>
    <td> <img src="Image-BK4819-Audio-Out-Mod.png" alt="BK4819 audio output modification of UV-K6" style="width: 800px;"/> </td>
  </tr>
</table>

## Reassembly

Re-assembly is the reverse of disassebly.

 1. Align the PCB with the metal frame.
 2. Replace the 5 screws holding the PCB to the frame.
 3. Fold the LCD back into place and align the tabs with the slots in the PCB.
 4. Snap the LCD into place.
 5. Tighten the nuts on the antenna connector and volume control potentiometer.
 6. Solder the speaker wires to the PCB.
 7. Ensure that the button pad is properly seated in the case.
 8. Place the plastic cover over the speaker-mic connector.
 9. Slide the frame into the case.
 10. Push the frame into the case until it snaps into place.
 11. Re-attach the knob, antenna and battery.

# Firmware

One can either build the firmware on one's own, or download a pre-built firmware file from the Mobilinkd repository.
Each release will have a `firmware.packed.bin` associated with it with the **Digital Modulation** feature enabled.

## Building From Source

Source code can be found here: https://github.com/mobilinkd/uv-k5-firmware-custom

The default firmware configuration has the `ENABLE_DIGITAL_MODULATION` option turned off.

Edit the `Makefile`. Find the entry for `ENABLE_DIGITAL_MODULATION` and set the value to 1.

I always enable `ENABLE_REDUCE_LOW_MID_TX_POWER` when testing because I either want extremely low power when
testing in the shack, or I want high power in the field. With this setting enabled, I never needed to use
a dummy load or an attentuator on the HT when testing.

On Linux, run `./compile-with-docker.sh` to build the firmware. A file called `firmware.packed.bin`
will be generated in the `compiled_firmware` directory.

## Flashing the Firmware

 1. Ensure the radio is turned off.
 2. Plug the programming cable into the computer.
 3. Plug the programming cable into the radio.
 4. Hold down the PTT button while turning on the radio.
 5. The flashlight LED on the radio will be lit when the radio is in programming mode.

<table>
  <tr>
    <td> <img src="UV-K6-programming.png" alt="UV-K6 in programming mode" style="width: 320px;"/> </td>
  </tr>
</table>


Use https://egzumer.github.io/uvtools/ to flash the firmware.

You can use this direct link to flash the pre-built binary: https://egzumer.github.io/uvtools/?firmwareURL=https://github.com/mobilinkd/uv-k5-firmware-custom/releases/download/v1be5a20/firmware-1be5a20.packed.bin

<table>
  <tr>
    <td> <img src="uvtools-screenshot.png" alt="uvtools programmer screenshot" style="width: 640px;"/> </td>
  </tr>
</table>

Once programmed, the new firmware will show on the boot screen.

<table>
  <tr>
    <td> <img src="UV-K6-boot-screen.png" alt="UV-K6 boot screen with new firmware" style="width: 320px;"/> </td>
  </tr>
</table>


## Configuring the Radio

When using digital modulation, all settings that can be disabled should be disabled. CTCSS, DCS,
VOX, any of the tail squelch elimination settings, power saver, etc. These should all be turned
off. The firmware should ignore most of these settings when in digital mode, but things like
power saver are not, and not all settings have been thoroughly tested at this point.

### Modulation

The main setting to change is the `Demodu` setting. This must be changed to `DIG`. You will see
`DIG` shown in place of `FM` in the display. This changes the modulation and demodulation mode
to `Digital Modulation`.

### VOX

VOX must be disabled. The pre-built firmare has the VOX feature disabled entirely so that there is
no possibility of accidentally enabling this. VOX turn-around time is far too slow for digital
communication.

### Wide vs. Narrow

The `W/N` setting should be set appropriately for the type of modulation. For 9600 baud GFSK, 
use `WIDE`. For 4-FSK modes, which have a narrower occupied bandwidth, use `NARROW`.

If the occupied bandwidth is less than 12.5kHz, use `NARROW`, otherwise use `WIDE`.

### Output Volume

Keep the output level low. The amplifier will easily overload many audio inputs. With the
UV-K6, the volume control knob has orange indicators (tick marks). Two ticks, or 1/3rd
rotation of the knob, is the maximum level that should be used. This is especially true when
using it with the Mobilinkd TNC3 or TNC4. Exceeding this can cause both demodulation issues
and behavioral problems with the TNC.

## Configuring the Mobilinkd TNC4

The hardware and firmware modifications were tested using a Mobilinkd TNC4. These recommended
settings are based on using the radio with a TNC4. Using it with another TNC or modem will
require calibration to determine the appropriate settings.

### Audio Output

The TNC4 should be configured for an output level of 190. This level has been calibrated in
the firmware to match the appropriate deviation for GMSK at 9600 baud when in `WIDE` mode
and proper deviation for M17 4-FSK when in `NARROW` mode.

### PTT Style

The TNC should be configured for `Simplex` PTT style.

### Audio Input

Audio input must be configured for a gain of 0.

### Modem Configuration

For M17 mode (or any 4-FSK mode), the `RX Reverse Polarity` must be enabled and `TX
Reverse Polarity` must be disabled.

# Results

The above modifications result in the ability to transmit and receive digital modes
such as 9600 baud GMSK and M17 4-FSK (4800 sym/s). It also reduces turn-around time by
almost half.

## Turn-around Time

Turn-around time is the time taken by the transceiver to switch from transmit to receive. It
is the recovery time of the receiver. A short turn-around time is necessary for efficient
digital communication.

These three images show the turn-around time for stock firmware (376ms), EGZUMER firmware (140ms), and
the new Mobilinkd (MOBLNKD) firmware (79ms).

<table>
  <tr>
    <td> <img src="Turn-around_time_stock.png" alt="Turn-around time of stock Quansheng firmware" style="width: 800px;"/> </td>
    <td> <img src="Turn-around_time_EGZUMER.png" alt="Turn-around time of EGZUMER firmware" style="width: 800px;"/> </td>
    <td> <img src="Turn-around_time_MOBLNKD.png" alt="Turn-around time of Mobilinkd firmware" style="width: 800px;"/> </td>
  </tr>
</table>

## Transmit Fidelity

This image shows the fidelity of a GFSK transmission, modulated by a Mobilinkd TNC4, transmitted by the UV-K6,
and received by an RTL-SDR is apparent. Both the low-frequency response and the high-frequency content, beyond
the 3kHz voice channel or even the BK4819's widest TX LPF (4.5kHz) is apparent.

<table>
  <tr>
    <td> <img src="UV-K6_GFSK_Transmit.png" alt="GFSK transmitted by UV-K6 and received by SDR" style="width: 800px;"/> </td>
  </tr>
</table>

## Receive Fidelity

This image shows an eye diagram of GFSK reception. The signal was transmitted by a PlutoSDR using GNU Radio,
and received on the UV-K6. This is the signal being sent from the UV-K6 to a Mobilinkd TNC4 for demodulation.

<table>
  <tr>
    <td> <img src="UV-K6_GFSK_Demod.png" alt="GFSK transmitted by UV-K6 and received by SDR" style="width: 800px;"/> </td>
  </tr>
</table>

# Future Work

Better RX performance and improved power efficiency can be acheived by completely bypassing the
audio amplifier when in digital mode. It may be possible to do this using a custom PCB similar to
what is done with the wide-band receiver modification. A couple of analog switches controlled by
the insertion of a connector in the speaker-mic jack would work. There is a switched terminal on
the 3.5mm jack which has 3.3V present when nothing is inserted into the jack.

The 1.4 version of the PCB would make this easy as it has a 0402 0 Ohm shunt to connect the audio
amp to the speaker-mic connector. The 1.5 version of the PCB would require modifying the PCB by
breaking a wide and easily accessible trace.

Turn-around time can be improved by changing the firmware to configure the transmitter and receiver
only when material changes are made. Currently, the firmware reconfigures many BK4819 parameters when
switching between transmit and receive. Some of this is likely done to make dual-receive simpler to
implement.

The transmit delay is the time between pressing PTT and the time the transmitter is active and
at full power. It is currently about 120ms. This is longer than requried.  The firmware does key
debounce which requires 30ms. This can be eliminated for PTT in `DIG` mode where PTT is controlled
by a modem/TNC. The same improvements for turn-around time will also reduce transmit delay. I suspect
it is possible to get each down to around 40ms without too much work.

The deviation values when in `DIG` mode for Wide and Narrow modes are hard-coded in the firmware
to values that work well for the TNC4. These should be moved into EEPROM settings to make it easier
to match the radio to other modems.

# Notes

## MIC Input Bias

 - The MIC input bias shoots up to ~3V at power-on. The nominal value for this is 1/2 the ADC voltage
(1.35V). The MIC ADC must be enabled (with RX DSP disabled -- see below) to bring the bias voltage
down to the proper level.
 - When MIC ADC is enabled (REG_30, bit 2), the DC bias is adjusted to the required level, but this takes
time based on the size of C77. At 10uF, this takes 2.5s. 1uF for C77 seems to decent low frequency
response and only requires 250ms to settle. This can be done at power-on in BK4819_Init().
 - Through a tedious process of elimination it was determined that when the RX DSP is enabled (REG_30,
bit 0), the MIC ADC bias level will not self-adjust. It will only adjust when MIC ADC is enabled
and RX DSP is disabled.
 - I suspect that the voltage is shooting up to 3V because the Egzumer firmware enables MIC AGC in
BK4819_Init() but I have not checked this yet. Worth investigating.

## Startup

 - The Egzumer startup processing seems like it could be optimized a bit. There are some time-consuming
BK4819 initialization which occurs before the welcome screen is shown. It seems that this could be done
while the welcome screen is shown.

## Mode Switching

 - Full testing of behavior when switching between `Demodu` settings has not been done. So far I
have not seen any issues when switching between `FM` and `DIG` modes.

## Build Script

 - I have added a build script for doing Docker builds on hosts with SELinux in enforcing mode.
 - The script includes support for adding the packed firmware artifact to the `Recent` file list.